In [9]:
import os
import math
from pathlib import Path
from dataclasses import dataclass
from typing import Optional, List

import pandas as pd
import geopandas as gpd
import numpy as np
import requests
from scipy.optimize import fsolve
from PIL import Image, ImageDraw

import ipywidgets as W
from ipyevents import Event
from IPython.display import display

# ============================================================
# CONFIG: EDIT THESE PATHS / KEY FOR YOUR ENVIRONMENT
# ============================================================

# !--- CSV of invalid cases (output from 0_filter.ipynb)
INVALID_CSV_PATH = Path("../YOUR/PATH/manual_collection/invalid_cases.csv")
invalid_df = pd.read_csv(INVALID_CSV_PATH)
print(f"Total rows to annotate: {len(invalid_df)}")

# !--- Input GeoJSON (which includes pano points along the road segment)
GEOJSON_PATH = Path("../YOUR/PATH/POINT_EPSG4326.geojson")

# !--- Directory containing original images from the automated tool
IMG_ROOT = Path("../YOUR/PATH/outputs_automation")

# !--- Directory where manual-annotation outputs (CSV + new images) will be saved
OUT_DIR = Path("../YOUR/PATH/manual_collection/outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# !--- Google Street View API key
# Needed only when zooming or moving to a pano that has not been downloaded yet.
API_KEY = "YOUR_API_KEY"

Total rows to annotate: 758


In [15]:
# ---------- CONFIG ----------
IMG_W = IMG_H = 640
CENTER_Y = 320

# Where manual annotations AND new downloads live
FINAL_CSV = OUT_DIR / "manual_collection_invalid.csv"

# cache directory for resized images (so they live under the notebook root)
CACHE_DIR = OUT_DIR / "_cache_images"
CACHE_DIR.mkdir(parents=True, exist_ok=True)

# geojson with all pano points along segments
PANOS_GDF = gpd.read_file(GEOJSON_PATH)

# ---------- GEOMETRY HELPERS (estimated width) ----------
def dist_from_center(y):
    """Convert a y-pixel to signed distance from the center line."""
    if y is None or (isinstance(y, float) and math.isnan(y)):
        return float("nan")
    return float(y - CENTER_Y)


def equations_top(T, p_0_top, p_10_top):
    if p_10_top < 0:
        return np.tan(np.radians(T)) / np.tan(np.radians(10 - T)) - (p_0_top / abs(p_10_top))
    else:
        return np.tan(np.radians(T)) / np.tan(np.radians(T - 10)) - (p_0_top / p_10_top)


def equations_bottom(T, p_0_bottom, p_10_bottom):
    if p_10_bottom < 0:
        return np.tan(np.radians(T)) / np.tan(np.radians(10 - T)) - (p_0_bottom / abs(p_10_bottom))
    else:
        return np.tan(np.radians(T)) / np.tan(np.radians(T - 10)) - (p_0_bottom / p_10_bottom)


def estimate_width_from_dists(p_0_top, p_10_top, p_0_bottom, p_10_bottom):
    if np.isnan(p_0_top) or np.isnan(p_10_top) or np.isnan(p_0_bottom) or np.isnan(p_10_bottom):
        return np.nan

    T_initial_top = 8 if p_10_top < 0 else 13
    T_initial_bottom = 8 if p_10_bottom < 0 else 13

    try:
        T_solution_top = fsolve(equations_top, T_initial_top, args=(p_0_top, p_10_top))
        T_solution_bottom = fsolve(equations_bottom, T_initial_bottom, args=(p_0_bottom, p_10_bottom))

        T_top = T_solution_top[0]
        T_bottom = T_solution_bottom[0]

        cot_T_top = 1 / np.tan(np.radians(T_top))
        cot_T_bottom = 1 / np.tan(np.radians(T_bottom))

        return 2.5 * (cot_T_top - cot_T_bottom)

    except Exception:
        return np.nan


def compute_estimated_width_row(row):
    return estimate_width_from_dists(
        row["top_dist_p0"],
        row["top_dist_p10"],
        row["bot_dist_p0"],
        row["bot_dist_p10"],
    )


# ---------- STREET VIEW HELPERS ----------
def get_streetview_image(panoid, heading, fov, pitch, side, save_root=OUT_DIR):
    """
    Download Google Street View image given panoid, heading, pitch, and side.
    """
    base_url = "https://maps.googleapis.com/maps/api/streetview"
    url = (
        f"{base_url}?size=640x640&pano={panoid}&heading={heading}"
        f"&fov={fov}&pitch={pitch}&source=outdoor&key={API_KEY}"
    )

    folder_path = save_root / str(panoid) / side
    folder_path.mkdir(parents=True, exist_ok=True)

    filename = f"pitch{pitch}_heading{heading}_fov{int(fov)}.jpg"
    file_path = folder_path / filename

    if file_path.exists():
        return str(file_path)

    response = requests.get(url)
    if response.status_code == 200:
        with open(file_path, "wb") as file:
            file.write(response.content)
        return str(file_path)
    else:
        print(f"Failed to download image (pitch={pitch}), Status Code: {response.status_code}")
        return None


def adjust_heading(pano_heading, bearing):
    """Adjust heading based on road bearing vs pano_heading."""
    diff = abs(pano_heading - bearing)
    diff = diff if diff <= 180 else 360 - diff  # shortest angular distance

    if diff <= 45 or diff >= 315:
        adjusted = pano_heading + 90
    else:
        adjusted = pano_heading - 90

    return adjusted % 360  # keep in [0, 360)


def ensure_images_for_pano(link_id, side, pano_id, pano_heading, bearing):
    """
    Make sure pitch0 and pitch-10 images exist for this pano.
    - First check in OUT_DIR (manual downloads)
    - Then in IMG_ROOT (original batch)
    - If missing, download into OUT_DIR using Street View API.
    """
    diff = abs(pano_heading - bearing)
    diff = diff if diff <= 180 else 360 - diff

    if diff <= 45 or diff >= 315:
        current_fov = 95
    else:
        current_fov = 65

    heading = adjust_heading(pano_heading, bearing)

    for pitch in (0, -10):
        img_path = find_image_path(pano_id, side, pitch)
        if img_path is None or not img_path.exists():
            saved = get_streetview_image(
                panoid=pano_id,
                heading=heading,
                fov=current_fov,
                pitch=pitch,
                side=side,
                save_root=OUT_DIR,
            )
            if saved:
                print(f"✅ Downloaded image for {link_id} {side} pano={pano_id}, pitch={pitch}")
            else:
                print(f"⚠️ Could not download image for {link_id} {side} pano={pano_id}, pitch={pitch}")


# ---------- DATA MODEL ----------
@dataclass
class RowState:
    link_id: str
    point_id: str
    side: str
    pano_id: str
    pano_heading: float
    original_width: float

    # current FOV
    fov: float = 65.0

    # clicked positions (x,y) in pixels (640x640)
    top_x_p0: float = float("nan")
    top_y_p0: float = float("nan")
    bot_x_p0: float = float("nan")
    bot_y_p0: float = float("nan")
    top_x_p10: float = float("nan")
    top_y_p10: float = float("nan")
    bot_x_p10: float = float("nan")
    bot_y_p10: float = float("nan")

    no_buffer: bool = False

    def to_record(self):
        # distances from center line (in pixels)
        top_dist_p0 = dist_from_center(self.top_y_p0)
        bot_dist_p0 = dist_from_center(self.bot_y_p0)
        top_dist_p10 = dist_from_center(self.top_y_p10)
        bot_dist_p10 = dist_from_center(self.bot_y_p10)

        return {
            "link_id": self.link_id,
            "point_id": self.point_id,
            "side": self.side,
            "pano_id": self.pano_id,
            "pano_heading": self.pano_heading,
            "original_width": self.original_width,
            "fov": self.fov,
            "top_x_p0": self.top_x_p0,
            "top_y_p0": self.top_y_p0,
            "bot_x_p0": self.bot_x_p0,
            "bot_y_p0": self.bot_y_p0,
            "top_x_p10": self.top_x_p10,
            "top_y_p10": self.top_y_p10,
            "bot_x_p10": self.bot_x_p10,
            "bot_y_p10": self.bot_y_p10,
            "top_dist_p0": top_dist_p0,
            "bot_dist_p0": bot_dist_p0,
            "top_dist_p10": top_dist_p10,
            "bot_dist_p10": bot_dist_p10,
            "no_buffer": bool(self.no_buffer),
        }


# ---------- IMAGE FINDER ----------
def find_image_path(pano_id: str, side: str, pitch: int, fov: Optional[float] = None) -> Optional[Path]:
    """
    Find the image file for a given pano/side/pitch/fov.

    Search order:
    1. OUT_DIR (manual downloads): manual_annotations_invalid/<pano_id>/<side>/
    2. IMG_ROOT (original batch outputs): /media/.../outputs/<pano_id>/<side>/
    """
    pitch_str = f"pitch{pitch}"

    for root in (OUT_DIR, IMG_ROOT):
        pano_dir = root / pano_id
        if not pano_dir.exists():
            continue

        side_dir = pano_dir / side
        if not side_dir.exists():
            continue

        # If FOV specified, look for exact match first
        if fov is not None:
            fov_int = int(round(fov))
            for img_file in side_dir.glob(f"{pitch_str}_heading*_fov{fov_int}.jpg"):
                s = str(img_file)
                if "_segmented.jpg" in s or "_final_lines.jpg" in s or "_road_lines.jpg" in s:
                    continue
                return img_file

        # Fall back to any matching pitch/heading (for old files without FOV)
        for img_file in side_dir.glob(f"{pitch_str}_heading*.jpg"):
            s = str(img_file)
            if "_segmented.jpg" in s or "_final_lines.jpg" in s or "_road_lines.jpg" in s:
                continue
            if "_fov" not in str(img_file):
                return img_file

    return None


# ---------- ANNOTATOR ----------
class InvalidSidewalkAnnotator:
    def __init__(self, df: pd.DataFrame, start_at: int = 0):
        if FINAL_CSV.exists():
            base_df = pd.read_csv(FINAL_CSV)
        else:
            base_df = df.copy()
            if "original_width" not in base_df.columns and "width" in base_df.columns:
                base_df["original_width"] = base_df["width"]

        self.rows: List[RowState] = []

        for _, r in base_df.iterrows():
            rs = RowState(
                link_id=str(r["link_id"]),
                point_id=str(r["point_id"]),
                side=str(r["side"]),
                pano_id=str(r["pano_id"]),
                pano_heading=float(r["pano_heading"]),
                original_width=float(r.get("original_width", r.get("width", np.nan))),
            )

            fov_val = r.get("fov", np.nan)
            if "fov" in base_df.columns and not pd.isna(fov_val):
                rs.fov = float(fov_val)
            else:
                try:
                    subset = PANOS_GDF[
                        (PANOS_GDF["link_id"] == rs.link_id)
                        & (PANOS_GDF["side"] == rs.side)
                        & (PANOS_GDF["pano_id"] == rs.pano_id)
                    ]
                    if not subset.empty:
                        pano_row = subset.iloc[0]
                        bearing = float(pano_row["bearing"])
                        diff = abs(rs.pano_heading - bearing)
                        diff = diff if diff <= 180 else 360 - diff
                        rs.fov = 95 if diff <= 45 or diff >= 315 else 65
                except Exception:
                    pass

            for fld in [
                "top_x_p0", "top_y_p0",
                "bot_x_p0", "bot_y_p0",
                "top_x_p10", "top_y_p10",
                "bot_x_p10", "bot_y_p10",
            ]:
                if fld in base_df.columns:
                    val = r.get(fld, np.nan)
                    try:
                        if not pd.isna(val):
                            setattr(rs, fld, float(val))
                    except Exception:
                        pass

            if "no_buffer" in base_df.columns:
                try:
                    rs.no_buffer = bool(r["no_buffer"])
                except Exception:
                    rs.no_buffer = False

            self.rows.append(rs)

        # start from the first image (row 0 or provided start_at)
        self.i = max(0, min(int(start_at), len(self.rows) - 1))
        self.pitch = 0  # 0 or -10

        # internal: current image src (served via /files/...)
        self._img_src: str = ""
        self._last_hover_y: float = CENTER_Y

        # UI components
        self.title = W.HTML()
        self.status = W.HTML()
        self.panel = W.HTML()
        self.panel.layout = W.Layout(
            width=f"{IMG_W}px",
            height=f"{IMG_H}px",
            display="inline-block",
            overflow="hidden",
            font_size="0px",
        )

        # Buttons
        self.btn_prev = W.Button(description="⬅ Prev row")
        self.btn_next = W.Button(description="Save & Next ➡", button_style="success")
        self.btn_pitch = W.ToggleButtons(
            options=[("Pitch 0", 0), ("Pitch -10", -10)],
            value=0,
        )
        self.btn_prev_pano = W.Button(description="Prev image point")
        self.btn_next_pano = W.Button(description="Next image point")
        self.btn_zoom_in = W.Button(description="Zoom in")
        self.btn_zoom_out = W.Button(description="Zoom out")
        self.btn_undo = W.Button(description="Undo last", button_style="warning")
        self.btn_reset = W.Button(description="Reset points", button_style="danger")
        self.btn_no_sw = W.Button(description="Mark 'No buffer'", button_style="info")
        self.btn_clear_sw = W.Button(description="Clear 'No buffer'")
        self.btn_next_unfinished = W.Button(description="Next un-annotated")
        self.btn_export = W.Button(description="Export CSV", button_style="primary")

        controls = W.HBox(
            [
                self.btn_prev,
                self.btn_next,
                self.btn_pitch,
                self.btn_prev_pano,
                self.btn_next_pano,
                self.btn_zoom_in,
                self.btn_zoom_out,
                self.btn_undo,
                self.btn_reset,
                self.btn_no_sw,
                self.btn_clear_sw,
                self.btn_next_unfinished,
                self.btn_export,
            ]
        )

        self.ui = W.VBox([self.title, self.panel, self.status, controls])

        # Mouse events
        self._ev = Event(source=self.panel, watched_events=["mousemove", "mouseleave", "click"])
        self._ev.on_dom_event(self._on_mouse_event)

        # Wire buttons
        self.btn_prev.on_click(self.on_prev)
        self.btn_next.on_click(self.on_next)
        self.btn_pitch.observe(self.on_pitch_change, names="value")
        self.btn_prev_pano.on_click(self.on_prev_pano)
        self.btn_next_pano.on_click(self.on_next_pano)
        self.btn_zoom_in.on_click(self.on_zoom_in)
        self.btn_zoom_out.on_click(self.on_zoom_out)
        self.btn_undo.on_click(self.on_undo)
        self.btn_reset.on_click(self.on_reset)
        self.btn_no_sw.on_click(self.on_no_sw)
        self.btn_clear_sw.on_click(self.on_clear_sw)
        self.btn_next_unfinished.on_click(self.on_next_unfinished)
        self.btn_export.on_click(self.on_export)

        # initial render
        self.render()

    # ---------- helper: does this row still need annotation? ----------
    def _needs_annotation(self, rs: RowState) -> bool:
        """
        Return True if this row does NOT yet have a valid estimated width and is NOT marked as 'No buffer'.
        """
        if rs.no_buffer:
            return False

        p0_top = dist_from_center(rs.top_y_p0)
        p10_top = dist_from_center(rs.top_y_p10)
        p0_bot = dist_from_center(rs.bot_y_p0)
        p10_bot = dist_from_center(rs.bot_y_p10)

        est_w = estimate_width_from_dists(p0_top, p10_top, p0_bot, p10_bot)
        return np.isnan(est_w)

    # ---------- IMAGE LOADING / CACHING ----------
    def _load_image_src(self) -> str:
        """
        Return the src string for the current image, using Jupyter's /files static handler.
        We cache a resized 640x640 PNG in CACHE_DIR and refer to it via /files/<relative_path>.
        """
        rs = self.rows[self.i]
        img_path = find_image_path(rs.pano_id, rs.side, self.pitch, rs.fov)

        if img_path is None or not img_path.exists():
            # Use a placeholder if nothing found
            placeholder = CACHE_DIR / "placeholder.png"
            if not placeholder.exists():
                im = Image.new("RGB", (IMG_W, IMG_H), (245, 245, 245))
                d = ImageDraw.Draw(im)
                d.text(
                    (20, 20),
                    f"Image not found\nPano: {rs.pano_id}\nSide: {rs.side}\nPitch: {self.pitch}",
                    fill=(60, 60, 60),
                )
                placeholder.parent.mkdir(parents=True, exist_ok=True)
                im.save(placeholder)
            cache_path = placeholder
        else:
            # Create (or reuse) a cached, resized PNG
            fov_key = "na"
            if rs.fov is not None and not (isinstance(rs.fov, float) and math.isnan(rs.fov)):
                fov_key = int(round(rs.fov))
            cache_name = f"{rs.pano_id}_{rs.side}_pitch{self.pitch}_fov{fov_key}.png"
            cache_path = CACHE_DIR / cache_name

            if not cache_path.exists():
                try:
                    im = Image.open(img_path)
                    if im.size != (IMG_W, IMG_H):
                        im = im.resize((IMG_W, IMG_H), Image.Resampling.LANCZOS)
                    cache_path.parent.mkdir(parents=True, exist_ok=True)
                    im.save(cache_path)
                except Exception as e:
                    # Fallback to an error placeholder
                    placeholder = CACHE_DIR / "error_placeholder.png"
                    if not placeholder.exists():
                        im = Image.new("RGB", (IMG_W, IMG_H), (245, 245, 245))
                        d = ImageDraw.Draw(im)
                        d.text((20, 20), f"Error loading image:\n{e}", fill=(60, 60, 60))
                        placeholder.parent.mkdir(parents=True, exist_ok=True)
                        im.save(placeholder)
                    cache_path = placeholder

        # IMPORTANT: build a /files/... URL using a path relative to the Jupyter server root
        # In your setup, the server root is typically your home directory.
        home = Path.home()
        rel = os.path.relpath(cache_path, start=home)
        src = f"/files/{rel}"
        return src


    # ---------- HTML/SVG composition ----------
    def _compose_html(self, hover_y: float) -> str:
        rs = self.rows[self.i]

        def dot(x, y, fill, stroke="#000"):
            if (
                x is None
                or y is None
                or (isinstance(x, float) and math.isnan(x))
                or (isinstance(y, float) and math.isnan(y))
            ):
                return ""
            return (
                f'<circle cx="{x:.1f}" cy="{y:.1f}" r="5" '
                f'fill="{fill}" stroke="{stroke}" stroke-width="2"/>'
            )

        if self.pitch == 0:
            top = dot(rs.top_x_p0, rs.top_y_p0, "cyan")
            bot = dot(rs.bot_x_p0, rs.bot_y_p0, "magenta")
        else:
            top = dot(rs.top_x_p10, rs.top_y_p10, "cyan")
            bot = dot(rs.bot_x_p10, rs.bot_y_p10, "magenta")

        nosw = ""
        if rs.no_buffer:
            nosw = (
                '<rect x="8" y="8" width="180" height="24" fill="white" opacity="0.7"/>'
                '<text x="16" y="26" fill="#c00" font-weight="bold">🚫 NO buffer</text>'
            )

        if hover_y is None:
            hover_y = -1000

        return f"""
<div id="svwrap" style="position:relative;width:{IMG_W}px;height:{IMG_H}px;line-height:0;">
  <img id="svimg"
       src="{self._img_src}"
       style="position:absolute;left:0;top:0;width:{IMG_W}px;height:{IMG_H}px;"/>
  <svg id="svsvg" xmlns="http://www.w3.org/2000/svg"
       width="{IMG_W}" height="{IMG_H}"
       style="position:absolute;left:0;top:0;pointer-events:none">
    <line x1="0" y1="{CENTER_Y}" x2="{IMG_W}" y2="{CENTER_Y}"
          stroke="yellow" stroke-opacity="0.7" stroke-width="2"/>
    <line x1="0" y1="{hover_y:.1f}" x2="{IMG_W}" y2="{hover_y:.1f}"
          stroke="lime" stroke-opacity="0.6" stroke-width="2"/>
    {top}
    {bot}
    {nosw}
  </svg>
</div>
        """

    # ---------- status ----------
    def _status_html(self, rs: RowState) -> str:
        def fmt_pair(t, b):
            def fmt(v):
                if v is None or (isinstance(v, float) and math.isnan(v)):
                    return "<b>NA</b>"
                return f"{v:.1f} (dist {v - CENTER_Y:+.1f})"
            return f"TOP: {fmt(t)} | BOTTOM: {fmt(b)}"

        p0_top = dist_from_center(rs.top_y_p0)
        p10_top = dist_from_center(rs.top_y_p10)
        p0_bot = dist_from_center(rs.bot_y_p0)
        p10_bot = dist_from_center(rs.bot_y_p10)
        est_w = estimate_width_from_dists(p0_top, p10_top, p0_bot, p10_bot)

        if np.isnan(est_w):
            est_line = "<b>Estimated width:</b> NA"
        else:
            est_line = f"<b>Estimated width:</b> {est_w:.2f} m"

        if rs.fov is None or (isinstance(rs.fov, float) and math.isnan(rs.fov)):
            fov_str = "NA"
        else:
            fov_str = f"{rs.fov:.1f}°"

        flag_str = " — <b style='color:#c00'>NO buffer</b>" if rs.no_buffer else ""
        return (
            f"{est_line}<br>"
            f"<b>FOV:</b> {fov_str}<br>"
            f"<b>link_id:</b> {rs.link_id} | <b>side:</b> {rs.side} | "
            f"<b>point_id:</b> {rs.point_id} | <b>pano:</b> {rs.pano_id}<br>"
            f"<b>Pitch 0</b> — {fmt_pair(rs.top_y_p0, rs.bot_y_p0)}<br>"
            f"<b>Pitch -10</b> — {fmt_pair(rs.top_y_p10, rs.bot_y_p10)}{flag_str}"
        )

    # ---------- render ----------
    def render(self):
        rs = self.rows[self.i]
        self._img_src = self._load_image_src()
        self.panel.value = self._compose_html(hover_y=self._last_hover_y)
        self.title.value = (
            f"<b>{self.i + 1}/{len(self.rows)}</b> — "
            f"link_id: <code>{rs.link_id}</code> | "
            f"side: <code>{rs.side}</code> | "
            f"pano: <code>{rs.pano_id}</code> | "
            f"pitch: <code>{self.pitch}</code>"
        )
        self.status.value = self._status_html(rs)

    # ---------- mouse ----------
    def _on_mouse_event(self, e: dict):
        et = e.get("type")
        rx = e.get("relativeX")
        ry = e.get("relativeY")

        def clamp(v, lo, hi):
            if v is None:
                return None
            v = float(v)
            if v < lo:
                v = lo
            if v > hi:
                v = hi
            return v

        rx = clamp(rx, 0, IMG_W - 1)
        ry = clamp(ry, 0, IMG_H - 1)

        if et == "mousemove" and ry is not None:
            if self._last_hover_y is not None and abs(self._last_hover_y - ry) < 1:
                return
            self._last_hover_y = ry
            self.panel.value = self._compose_html(hover_y=ry)

        elif et == "mouseleave":
            self._last_hover_y = None
            self.panel.value = self._compose_html(hover_y=None)

        elif et == "click" and rx is not None and ry is not None:
            self._handle_click(rx, ry)

    # ---------- pano navigation helpers ----------
    def _switch_to_pano_row(self, pano_row):
        """Update current RowState to use a new pano from PANOS_GDF."""
        rs = self.rows[self.i]

        new_pano = str(pano_row["pano_id"])
        new_heading = float(pano_row["pano_heading"])
        new_bearing = float(pano_row["bearing"])
        new_point_id = int(pano_row["point_id"])
        side = str(pano_row["side"])

        diff = abs(new_heading - new_bearing)
        diff = diff if diff <= 180 else 360 - diff
        rs.fov = 95 if diff <= 45 or diff >= 315 else 65

        ensure_images_for_pano(
            link_id=rs.link_id,
            side=side,
            pano_id=new_pano,
            pano_heading=new_heading,
            bearing=new_bearing,
        )

        rs.pano_id = new_pano
        rs.pano_heading = new_heading
        rs.point_id = str(new_point_id)
        rs.side = side

        rs.top_x_p0 = rs.top_y_p0 = rs.bot_x_p0 = rs.bot_y_p0 = float("nan")
        rs.top_x_p10 = rs.top_y_p10 = rs.bot_x_p10 = rs.bot_y_p10 = float("nan")
        rs.no_buffer = False

        self._save()
        self._last_hover_y = CENTER_Y
        self.render()

    def on_next_pano(self, _):
        rs = self.rows[self.i]
        try:
            curr_pid = int(rs.point_id)
        except Exception:
            print("Current point_id is not an integer; cannot move to next pano.")
            return

        subset = PANOS_GDF[
            (PANOS_GDF["link_id"] == rs.link_id) & (PANOS_GDF["side"] == rs.side)
        ].sort_values("point_id")

        target_pid = curr_pid + 1
        cand = subset[subset["point_id"] == target_pid]
        if cand.empty:
            print("No next image point for this link/side.")
            return

        self._switch_to_pano_row(cand.iloc[0])

    def on_prev_pano(self, _):
        rs = self.rows[self.i]
        try:
            curr_pid = int(rs.point_id)
        except Exception:
            print("Current point_id is not an integer; cannot move to previous pano.")
            return

        subset = PANOS_GDF[
            (PANOS_GDF["link_id"] == rs.link_id) & (PANOS_GDF["side"] == rs.side)
        ].sort_values("point_id")

        target_pid = curr_pid - 1
        cand = subset[subset["point_id"] == target_pid]
        if cand.empty:
            print("No previous image point for this link/side.")
            return

        self._switch_to_pano_row(cand.iloc[0])

    # ---------- zoom helpers ----------
    def _update_zoom(self, delta_fov: float):
        rs = self.rows[self.i]

        subset = PANOS_GDF[
            (PANOS_GDF["link_id"] == rs.link_id)
            & (PANOS_GDF["side"] == rs.side)
            & (PANOS_GDF["pano_id"] == rs.pano_id)
        ]
        if subset.empty:
            print("No geojson entry for this pano; cannot adjust FOV.")
            return

        pano_row = subset.iloc[0]
        bearing = float(pano_row["bearing"])
        pano_heading = float(pano_row["pano_heading"])

        if rs.fov is None or (isinstance(rs.fov, float) and math.isnan(rs.fov)):
            diff = abs(pano_heading - bearing)
            diff = diff if diff <= 180 else 360 - diff
            rs.fov = 95 if diff <= 45 or diff >= 315 else 65

        new_fov = rs.fov + delta_fov
        new_fov = max(30, min(120, new_fov))
        rs.fov = new_fov

        heading = adjust_heading(pano_heading, bearing)

        for pitch in (0, -10):
            saved = get_streetview_image(
                panoid=rs.pano_id,
                heading=heading,
                fov=new_fov,
                pitch=pitch,
                side=rs.side,
                save_root=OUT_DIR,
            )
            if saved:
                print(f"Updated zoom FOV={new_fov} for pano={rs.pano_id}, pitch={pitch}")

        self._save()
        self._last_hover_y = CENTER_Y
        self.render()

    def on_zoom_in(self, _):
        # Zoom in: smaller FOV
        self._update_zoom(-10)

    def on_zoom_out(self, _):
        # Zoom out: larger FOV
        self._update_zoom(+10)

    # ---------- clicks & row buttons ----------
    def _handle_click(self, x, y):
        rs = self.rows[self.i]
        if rs.no_buffer:
            return

        if self.pitch == 0:
            if math.isnan(rs.top_y_p0):
                rs.top_x_p0, rs.top_y_p0 = x, y
            elif math.isnan(rs.bot_y_p0):
                rs.bot_x_p0, rs.bot_y_p0 = x, y
        else:
            if math.isnan(rs.top_y_p10):
                rs.top_x_p10, rs.top_y_p10 = x, y
            elif math.isnan(rs.bot_y_p10):
                rs.bot_x_p10, rs.bot_y_p10 = x, y

        self._save()
        self.panel.value = self._compose_html(hover_y=self._last_hover_y)
        self.status.value = self._status_html(rs)

    def on_pitch_change(self, change):
        self.pitch = change["new"]
        self._last_hover_y = CENTER_Y
        self.render()

    def on_undo(self, _):
        rs = self.rows[self.i]
        if self.pitch == 0:
            if not math.isnan(rs.bot_y_p0):
                rs.bot_x_p0 = rs.bot_y_p0 = float("nan")
            elif not math.isnan(rs.top_y_p0):
                rs.top_x_p0 = rs.top_y_p0 = float("nan")
        else:
            if not math.isnan(rs.bot_y_p10):
                rs.bot_x_p10 = rs.bot_y_p10 = float("nan")
            elif not math.isnan(rs.top_y_p10):
                rs.top_x_p10 = rs.top_y_p10 = float("nan")
        self._save()
        self.render()

    def on_reset(self, _):
        rs = self.rows[self.i]
        if self.pitch == 0:
            rs.top_x_p0 = rs.top_y_p0 = rs.bot_x_p0 = rs.bot_y_p0 = float("nan")
        else:
            rs.top_x_p10 = rs.top_y_p10 = rs.bot_x_p10 = rs.bot_y_p10 = float("nan")
        self._save()
        self.render()

    def on_no_sw(self, _):
        rs = self.rows[self.i]
        rs.no_buffer = True
        rs.top_x_p0 = rs.top_y_p0 = rs.bot_x_p0 = rs.bot_y_p0 = float("nan")
        rs.top_x_p10 = rs.top_y_p10 = rs.bot_x_p10 = rs.bot_y_p10 = float("nan")
        self._save()
        self.render()

    def on_clear_sw(self, _):
        self.rows[self.i].no_buffer = False
        self._save()
        self.render()

    def on_prev(self, _):
        if self.i > 0:
            self.i -= 1
            self._last_hover_y = CENTER_Y
            self.render()

    def on_next(self, _):
        self._save()
        if self.i < len(self.rows) - 1:
            self.i += 1
        self._last_hover_y = CENTER_Y
        self.render()

    def on_next_unfinished(self, _):
        """
        Jump to the next row (cyclically) that still needs annotation:
        - estimated_width is NaN
        - no_buffer is False
        """
        if not self.rows:
            return

        start = self.i
        n = len(self.rows)

        for offset in range(1, n + 1):
            idx = (start + offset) % n
            rs = self.rows[idx]
            if self._needs_annotation(rs):
                self.i = idx
                self._last_hover_y = CENTER_Y
                self.render()
                return

        print("All rows either have an estimated width or are marked 'No buffer'.")

    def on_export(self, _):
        self._save()
        print(f"Exported to {FINAL_CSV}")

    # ---------- save ----------
    def _save(self):
        df = pd.DataFrame([r.to_record() for r in self.rows])

        # add estimated_width column using current annotations
        df["estimated_width"] = df.apply(compute_estimated_width_row, axis=1)

        df.to_csv(FINAL_CSV, index=False)


# ---------- LAUNCH ----------
annotator = InvalidSidewalkAnnotator(invalid_df, start_at=0)

print(f"Loaded {len(invalid_df)} rows for annotation")
unfinished_count = sum(
    1 for rs in annotator.rows
    if annotator._needs_annotation(rs)
)
print(f"Un-annotated rows remaining: {unfinished_count}")
print(f"Existing images root: {IMG_ROOT}")
print(f"New downloads + CSV in: {OUT_DIR}")
print(f"Output CSV will be: {FINAL_CSV}")

display(annotator.ui)

Loaded 758 rows for annotation
Un-annotated rows remaining: 84
Existing images root: /media/cura/slieu3/CS_street_buffer/CS_ver3/outputs
New downloads + CSV in: /home/slieu3/CS_street_buffer/complete_street/v3/manual_annotation/outputs
Output CSV will be: /home/slieu3/CS_street_buffer/complete_street/v3/manual_annotation/outputs/manual_collection_invalid.csv


/tmp/ipykernel_3007872/1327059261.py:46: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  T_solution_bottom = fsolve(equations_bottom, T_initial_bottom, args=(p_0_bottom, p_10_bottom))
